In [1]:
from generator_based_on_Seattle import *
from parameters import *
from datetime import datetime

In [2]:
# generate traffic 

generate_traffic_based_on_seattle(
    from_date=FROM_DATE, 
    to_date=TO_DATE, 
    first_erlang=LAMBDA_0, 
    constant_bitrate=IS_CONSTANT_BITRATE, 
    aggregation=AGGREGATION, 
    resampling=RESAMPLING,
    period_length=UPGRADE_PERIOD, 
    actual_traffic_folder_path="real")

In [3]:
# predict traffic

time_delta = datetime.strptime(TO_DATE,"%Y-%m-%d") - datetime.strptime(FROM_DATE,"%Y-%m-%d")
n_periods = math.floor(time_delta.days / UPGRADE_PERIOD)

for current_period_start in range(0, n_periods*UPGRADE_PERIOD, UPGRADE_PERIOD):
    perdict_traffic_for_next_period(pd.read_csv(f"real/actual_traffic_{current_period_start}_{current_period_start+UPGRADE_PERIOD}.csv"),predicted_traffic_alphafolder_path="predicted",index=current_period_start)

In [4]:
n_connections_differences_overall = []
mean_holding_time_differences_overall = []

for current_period_start in range(0 + UPGRADE_PERIOD, n_periods*UPGRADE_PERIOD, UPGRADE_PERIOD):
    print(current_period_start)
    real_traffic = pd.read_csv(f"real/actual_traffic_{current_period_start}_{current_period_start+UPGRADE_PERIOD}.csv")
    real_traffic['holding_time'] = real_traffic['departure_time'] - real_traffic['arrival_time']
    predicted_traffic = pd.read_csv(f"predicted/predicted_traffic_{current_period_start}_{current_period_start+UPGRADE_PERIOD}.csv")
    predicted_traffic['holding_time'] = predicted_traffic['departure_time'] - predicted_traffic['arrival_time']

    n_connections_generated = []
    n_connections_predicted = []
    n_connections_differences = []
    mean_holding_time_generated = []
    mean_holding_time_predicted = []
    mean_holding_time_differences = []
    for source_node in range(14):
        for destination_node in range(14):
            if(source_node != destination_node):
                generated_connections = real_traffic.loc[(real_traffic['source_id'] == source_node) & (real_traffic['destination_id'] == destination_node)]
                predicted_connections = predicted_traffic.loc[(predicted_traffic['source_id'] == source_node) & (predicted_traffic['destination_id'] == destination_node)]
                generated_mean_holding_time = generated_connections['holding_time'].mean()
                predicted_mean_holding_time = predicted_connections['holding_time'].mean()
                if(math.isnan(generated_mean_holding_time)):
                    print("{} --> {}".format(source_node,destination_node))
                if(math.isnan(predicted_mean_holding_time)):
                    print("{} --> {}".format(source_node,destination_node))    

                n_connections_generated.append(len(generated_connections))
                n_connections_predicted.append(len(predicted_connections))
                n_connections_differences.append(len(predicted_connections) - len(generated_connections))
                mean_holding_time_generated.append(generated_mean_holding_time)
                mean_holding_time_predicted.append(predicted_mean_holding_time)
                mean_holding_time_differences.append(predicted_mean_holding_time-generated_mean_holding_time)

    n_connections_differences_overall.append(n_connections_differences)
    mean_holding_time_differences_overall.append(mean_holding_time_differences)            


90
180
270
360
450
540
630
720
810
900
990
1080
1170
1260
1350
1440
1530
1620
1710
1800
1890
1980
2070
2160
2250
2340
2430
2520
2610
2700
2790
2880
2970
3060
3150
3240
3330
3420
3510


In [5]:
for i in range(len(n_connections_differences_overall)):
    print(f"n_connections diff {np.mean(n_connections_differences_overall[i])}, holding_time diff {np.mean(mean_holding_time_differences_overall[i])}")

n_connections diff 19.554945054945055, holding_time diff 0.33283103114374246
n_connections diff 180.80769230769232, holding_time diff 0.38747342327489503
n_connections diff 136.06043956043956, holding_time diff 0.3192025577255367
n_connections diff -17.934065934065934, holding_time diff 0.4876121536345657
n_connections diff 316.24725274725273, holding_time diff 0.3204720725987485
n_connections diff -9.32967032967033, holding_time diff 0.21392044809369348
n_connections diff 387.9175824175824, holding_time diff 0.38595505697327925
n_connections diff 185.85164835164835, holding_time diff 0.5640766603796024
n_connections diff 463.7032967032967, holding_time diff 0.2451066909303473
n_connections diff 125.72527472527473, holding_time diff 0.29820733289728857
n_connections diff 282.9175824175824, holding_time diff 0.1132621825730397
n_connections diff 314.8076923076923, holding_time diff 0.284296522569384
n_connections diff 261.5934065934066, holding_time diff 0.4229087899310017
n_connections